In [1]:
# Import libraries
import numpy as np
import pandas as pd

# Import data
frauds = pd.read_csv('data/frauds.csv')
nonfrauds = pd.read_csv('data/nonfrauds.csv')

/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (8,12) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [8]:
Freshfrauds = pd.read_csv('data/FreshFrauds.csv')

def ToDate(date):
    s = str(date[-4:] + "/" + date[:-5])
    return s

def ToAmount(n):
    return n[1:]

# Remove Na date
Freshfrauds = Freshfrauds[Freshfrauds['Date of Txn'].notna()]

# Remove all inappropriate dates
for date in Freshfrauds['Date of Txn']:
    if type(date) == str:
        if date[-5] != "/":
            Freshfrauds = Freshfrauds[Freshfrauds['Date of Txn'] != date]
            
# Change date to appropriate format and comvert it to datetime format
Freshfrauds['Date of Txn'] = Freshfrauds['Date of Txn'].apply(ToDate)
Freshfrauds['cc_amount'] = Freshfrauds['Amount'].apply(ToAmount)
Freshfrauds['user_transaction_time'] = pd.to_datetime(Freshfrauds['Date of Txn'])

# Modify Columns
Freshfrauds['key'] = Freshfrauds['Supplier Key']
Freshfrauds['transaction_risk_score'] = Freshfrauds['Risk Score']
Freshfrauds = Freshfrauds.drop(['Supplier Name','Customer First Name','Customer Last Name','Merchant','Month of Disputed Txns',
                 'Date Dispute Initated','Comment','Marqeta Dispute Decision','Auth ID','MID','Temporary Credit Amount',
                 'Temporary credit reversed','Logged by','Supplier Key','Amount','Risk Score'], axis=1)

unadded_columns = ['channel','processing_type','ledger_balance','cc_acceptor_state','acceptor_state','cc_acceptor_country',
                   'cardholder_presence','card_presence','partial_approval_capable','pin_present']

for col in unadded_columns:
    Freshfrauds[col] = np.nan
    

    
Freshfrauds = Freshfrauds[['key','user_transaction_time','channel','transaction_risk_score','processing_type','cc_amount',
                          'ledger_balance','cc_acceptor_state','acceptor_state','cc_acceptor_country','cardholder_presence',
                          'card_presence','partial_approval_capable','pin_present']]

# Combine the frauds
df = frauds.append(Freshfrauds)

# Export data
df.to_csv('data/updatedfrauds.csv')

In [9]:
df

,key,user_transaction_time,channel,transaction_risk_score,processing_type,cc_amount,ledger_balance,cc_acceptor_state,acceptor_state,cc_acceptor_country,cardholder_presence,card_presence,partial_approval_capable,pin_present
0,9d851f1b-00b3-4b02-9494-beda8866d6f7,2020-12-29T16:13:49Z,ECOMMERCE,82.0,MANUAL,14.99,134.91,NaN,NaN,HK,0.0,0.0,0.0,False
1,dc13ebc6-a527-46d0-b424-6642d47c8a11,2020-12-29T16:13:32Z,ECOMMERCE,82.0,MANUAL,14.99,359.76,NaN,NaN,HK,0.0,0.0,0.0,False
2,5e8f4d8f-a720-4a3d-ac55-f8953db3b527,2020-12-29T16:13:13Z,ECOMMERCE,82.0,MANUAL,14.99,419.72,NaN,NaN,HK,0.0,0.0,0.0,False
3,74e6ff6d-f341-4300-8cac-30ab9e5ed5cf,2020-12-29T16:12:56Z,ECOMMERCE,82.0,MANUAL,14.99,149.90,NaN,NaN,HK,0.0,0.0,0.0,False
4,c63d2508-f935-4092-bde1-434e1acbd0ef,2020-12-29T16:12:38Z,ECOMMERCE,82.0,MANUAL,14.99,164.89,NaN,NaN,HK,0.0,0.0,0.0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
833,78d51c06-355b-4b40-b366-896fd223c356,2022-03-01 00:00:00,NaN,NaN,NaN,200.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
834,c25cd8f8-ad57-4206-85e1-04232ea57abd,2022-01-16 00:00:00,NaN,NaN,NaN,45.95,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
835,c25cd8f8-ad57-4206-85e1-04232ea57abd,2022-02-03 00:00:00,NaN,NaN,NaN,32.03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
836,c25cd8f8-ad57-4206-85e1-04232ea57abd,2022-02-20 00:00:00,NaN,NaN,NaN,26.14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
# Import data
frauds = pd.read_csv('data/updatedfrauds.csv')
frauds = frauds.drop(frauds.columns[[0]], axis = 1)
frauds = frauds[frauds['user_transaction_time'].notna()]
nonfrauds = pd.read_csv('data/nonfrauds.csv')

# Convert datetime and label data 
frauds['user_transaction_time'] = pd.to_datetime(frauds['user_transaction_time'], errors='coerce', utc=True)
frauds['date'] = frauds['user_transaction_time'].dt.date

nonfrauds['user_transaction_time'] = pd.to_datetime(nonfrauds['user_transaction_time'])
nonfrauds['date'] = nonfrauds['user_transaction_time'].dt.date

frauds['is_fraud'] = 1
nonfrauds['is_fraud'] = 0

# Sampling for each day by taking 4% of good transactions and 100% of frauds
frauds_date = sorted(list(set(frauds['date'])))[4:]
nonfraud = pd.DataFrame()

for date in frauds_date:
    dftemp = nonfrauds.loc[nonfrauds['date'] == date]
    dftemp = dftemp.sample(frac = 0.04, random_state = 0)
    nonfraud = nonfraud.append(dftemp)

df2 = frauds.append(nonfraud)
df2 = df2.sort_values(by='date')
df2 = df2[df2['date'] >= pd.Timestamp(2018,1,1)]
df2 = df2.drop(['key'], axis=1)
df2.to_csv('data/LabeledTransactions.csv')

/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (8,12) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [11]:
df2

,user_transaction_time,channel,transaction_risk_score,processing_type,cc_amount,ledger_balance,cc_acceptor_state,acceptor_state,cc_acceptor_country,cardholder_presence,card_presence,partial_approval_capable,pin_present,date,is_fraud
2826821,2018-12-25 02:52:37+00:00,OTHER,30.0,CHIP,114.08,57.13,PA,42,US,1.0,1.0,1.0,False,2018-12-25,0
2682222,2018-12-25 02:08:17+00:00,ECOMMERCE,26.0,MANUAL,34.66,3.22,NY,36,US,0.0,0.0,0.0,False,2018-12-25,0
660477,2018-12-25 07:28:11+00:00,ECOMMERCE,33.0,MANUAL,39,3767.11,AR,5,US,0.0,0.0,0.0,False,2018-12-25,0
1262291,2018-12-25 05:22:31+00:00,OTHER,28.0,CHIP,5.97,0.00,CA,6,US,1.0,1.0,1.0,False,2018-12-25,0
2960302,2018-12-25 21:05:55+00:00,OTHER,26.0,MAGSTRIPE,31.33,49.95,TX,48,US,1.0,1.0,1.0,False,2018-12-25,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1418639,2022-03-01 17:28:36+00:00,ECOMMERCE,1.0,CARD_ON_FILE,10,171.56,DC,11,US,0.0,0.0,0.0,False,2022-03-01,0
2505945,2022-03-01 18:23:15+00:00,OTHER,4.0,CHIP,99.1,237.64,NJ,34,US,1.0,1.0,1.0,False,2022-03-01,0
2644136,2022-03-01 18:03:28+00:00,OTHER,3.0,CHIP,130.49,0.00,NY,36,US,1.0,1.0,1.0,False,2022-03-01,0
1527533,2022-03-01 16:02:09+00:00,OTHER,1.0,CHIP,29.67,206.69,FL,12,US,1.0,1.0,1.0,False,2022-03-01,0


In [6]:
# Convert datetime and label data 
frauds = pd.read_csv('data/frauds.csv')
frauds['user_transaction_time'] = pd.to_datetime(frauds['user_transaction_time'], errors='coerce', utc=True)
frauds['date'] = frauds['user_transaction_time'].dt.date

nonfrauds['user_transaction_time'] = pd.to_datetime(nonfrauds['user_transaction_time'])
nonfrauds['date'] = nonfrauds['user_transaction_time'].dt.date

frauds['is_fraud'] = 1
nonfrauds['is_fraud'] = 0

# Sampling for each day by taking 4% of good transactions and 100% of frauds
frauds_date = sorted(list(set(frauds['date'])))[4:]
nonfraud = pd.DataFrame()

for date in frauds_date:
    dftemp = nonfrauds.loc[nonfrauds['date'] == date]
    dftemp = dftemp.sample(frac = 0.04, random_state = 0)
    nonfraud = nonfraud.append(dftemp)

df2 = frauds.append(nonfraud)
df2 = df2.sort_values(by='date')
df2 = df2[df2['date'] >= pd.Timestamp(2018,1,1)]
df2 = df2.drop(['key'], axis=1)
df2.to_csv('data/LabeledTransactions2.csv')

In [7]:
df2

,user_transaction_time,channel,transaction_risk_score,processing_type,cc_amount,ledger_balance,cc_acceptor_state,acceptor_state,cc_acceptor_country,cardholder_presence,card_presence,partial_approval_capable,pin_present,date,is_fraud
391,2018-12-25 12:30:50+00:00,ECOMMERCE,64.0,MANUAL,220.00,1270.96,NJ,34,US,0.0,0.0,0.0,False,2018-12-25,1
392,2018-12-25 12:22:59+00:00,ECOMMERCE,61.0,MANUAL,220.00,1050.96,NJ,34,US,0.0,0.0,0.0,False,2018-12-25,1
393,2018-12-25 12:08:10+00:00,ECOMMERCE,86.0,MANUAL,195.00,2100.96,NJ,34,US,0.0,0.0,0.0,False,2018-12-25,1
394,2018-12-25 11:12:46+00:00,ECOMMERCE,43.0,MANUAL,220.00,1490.96,NJ,34,US,0.0,0.0,0.0,False,2018-12-25,1
390,2019-01-27 21:00:20+00:00,ECOMMERCE,98.0,MANUAL,1426.60,4374.92,NaN,NaN,SK,0.0,0.0,0.0,False,2019-01-27,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
567381,2020-12-29 20:30:32+00:00,ECOMMERCE,8.0,CARD_ON_FILE,77.41,768.94,AR,5,US,0.0,0.0,0.0,False,2020-12-29,0
2201850,2020-12-29 00:34:45+00:00,ECOMMERCE,4.0,MANUAL,52.38,7125.71,IL,17,US,0.0,0.0,0.0,False,2020-12-29,0
568249,2020-12-29 04:40:45+00:00,ECOMMERCE,1.0,MANUAL,9.77,12826.93,AR,5,US,0.0,0.0,0.0,False,2020-12-29,0
1674866,2020-12-29 18:38:56+00:00,ECOMMERCE,17.0,MANUAL,32.78,11564.87,FL,12,US,0.0,0.0,0.0,False,2020-12-29,0
